In [2]:
import numpy as np
import spreg
import libpysal
import bambi as bmb
import arviz as az
import pandas as pd
import geopandas
import numpy
from pysal.lib import weights
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from pysal.lib import cg as geometry

from spreg import make_x, make_xb, dgp_lag


ModuleNotFoundError: No module named 'spreg'

In [ ]:
# Get points in a grid
l = np.arange(5)
xs, ys = np.meshgrid(l, l)
# Set up store
polys = []
# Generate polygons
for x, y in zip(xs.flatten(), ys.flatten()):
    poly = Polygon([(x, y), (x + 1, y), (x + 1, y + 1), (x, y + 1)])
    polys.append(poly)
# Convert to GeoSeries
polys = geopandas.GeoSeries(polys)
gdf = geopandas.GeoDataFrame(
    {
        "geometry": polys,
        "id": ["P-%s" % str(i).zfill(2) for i in range(len(polys))],
    }
)
# Build a rook contiguity matrix from a regular 3x3
# lattice stored in a geo-table
wr = weights.contiguity.Rook.from_dataframe(gdf)

In [ ]:
pd.DataFrame(*wr.full()).astype(int)

In [ ]:
rng = np.random.default_rng(12345)
rho = .8

u = make_x(rng,25,mu=[0],varu=[1], method='normal')

x = make_x(rng,25,mu=[0],varu=[1])

xb = make_xb(x,[1,2])
xb2 = xb
# w  = libpysal.weights.lat2W(5, 5)

wr.transform = "r"

y_d = dgp_lag(u, xb, wr,rho=rho)
temp1 = weights.lag_spatial(wr,y_d)

In [ ]:
rng = np.random.default_rng(243)

u = make_x(rng,25,mu=[0],varu=[1], method='normal')

x = make_x(rng,25,mu=[0],varu=[1])

xb = make_xb(x,[1,2])

# w  = libpysal.weights.lat2W(5, 5)

wr.transform = "r"
y_d2 = dgp_lag(u, xb, wr, rho=rho)
y_d = np.append(y_d,y_d2)
xb = np.append(xb2, xb)
tmp2 = weights.lag_spatial(wr,y_d2)
data_y = np.append(temp1,tmp2)

In [ ]:
y_d = y_d.reshape(-1,1)
xb = xb.reshape(-1,1)

In [ ]:
type(wr)

In [ ]:
# weights.lag_spatial(wr,y_d)

In [ ]:
# w  = libpysal.weights.lat2W(5, 5)
# wr = weights.contiguity.Rook.from_dataframe(gdf)
fe_lag = spreg.Panel_FE_Lag(y=y_d, x=xb, w=wr)
print(fe_lag.summary)

In [ ]:
df = pd.DataFrame(y_d, columns=["y"])
df["w_y"] = data_y
df["x"] = xb
model = bmb.Model(
    "y ~ x + w_y",
    df, dropna=True
)
results = model.fit()

In [ ]:
az.plot_trace(
    results,
    compact=True,
)

In [ ]:
az.summary(results)